# Fire Detection Model Training
This notebook trains a CNN model to detect fire in images and converts it to TensorFlow Lite for mobile deployment.

## 1. Setup and Install Dependencies

In [ ]:
# Install required packages
!pip install tensorflow
!pip install pillow
!pip install matplotlib
!pip install scikit-learn
!pip install opencv-python

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

## 2. Extract and Prepare Dataset

In [ ]:
# Extract the dataset (assuming Firetrain.zip is uploaded to Colab)
with zipfile.ZipFile('Firetrain.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

# Check dataset structure
!ls -la dataset/
print("\nDataset structure:")
!find dataset/ -type d

In [ ]:
# Count images in each category
fire_dir = 'dataset/fire_dataset/fire_images'
non_fire_dir = 'dataset/fire_dataset/non_fire_images'

fire_count = len(os.listdir(fire_dir))
non_fire_count = len(os.listdir(non_fire_dir))

print(f"Fire images: {fire_count}")
print(f"Non-fire images: {non_fire_count}")
print(f"Total images: {fire_count + non_fire_count}")
print(f"Fire ratio: {fire_count / (fire_count + non_fire_count) * 100:.1f}%")

## 3. Data Preprocessing and Visualization

In [ ]:
# Image parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

def load_and_preprocess_image(image_path, label):
    """Load and preprocess a single image"""
    try:
        image = tf.io.read_file(image_path)
        image = tf.image.decode_image(image, channels=3, expand_animations=False)
        
        # Ensure the image is 3D (height, width, channels)
        if len(image.shape) > 3:
            image = image[0]  # Take first frame if animated
        
        image.set_shape([None, None, 3])  # Set shape explicitly
        image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
        image = tf.cast(image, tf.float32) / 255.0
        return image, label
    except:
        # Return a black image if there's an error
        image = tf.zeros([IMG_SIZE, IMG_SIZE, 3], dtype=tf.float32)
        return image, label

# Filter out problematic files first
def is_valid_image(file_path):
    """Check if file is a valid static image"""
    try:
        with Image.open(file_path) as img:
            return img.format in ['JPEG', 'PNG', 'JPG'] and not getattr(img, 'is_animated', False)
    except:
        return False

# Create file paths and labels with filtering
fire_files = [os.path.join(fire_dir, f) for f in os.listdir(fire_dir) 
              if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
non_fire_files = [os.path.join(non_fire_dir, f) for f in os.listdir(non_fire_dir) 
                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Filter out problematic images
fire_files = [f for f in fire_files if is_valid_image(f)]
non_fire_files = [f for f in non_fire_files if is_valid_image(f)]

# Create labels (1 for fire, 0 for non-fire)
fire_labels = [1] * len(fire_files)
non_fire_labels = [0] * len(non_fire_files)

all_files = fire_files + non_fire_files
all_labels = fire_labels + non_fire_labels

print(f"Valid fire images: {len(fire_files)}")
print(f"Valid non-fire images: {len(non_fire_files)}")
print(f"Total valid files: {len(all_files)}")
print(f"Total labels: {len(all_labels)}")

In [ ]:
# Visualize sample images
def show_sample_images(files, labels, class_names=['Non-Fire', 'Fire']):
    fig, axes = plt.subplots(2, 4, figsize=(15, 8))
    
    # Show fire images
    fire_samples = [f for f, l in zip(files, labels) if l == 1][:4]
    for i, img_path in enumerate(fire_samples):
        img = Image.open(img_path)
        axes[0, i].imshow(img)
        axes[0, i].set_title('Fire')
        axes[0, i].axis('off')
    
    # Show non-fire images
    non_fire_samples = [f for f, l in zip(files, labels) if l == 0][:4]
    for i, img_path in enumerate(non_fire_samples):
        img = Image.open(img_path)
        axes[1, i].imshow(img)
        axes[1, i].set_title('Non-Fire')
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()

show_sample_images(all_files, all_labels)

## 4. Create TensorFlow Dataset

In [ ]:
# Split the data
train_files, val_files, train_labels, val_labels = train_test_split(
    all_files, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

print(f"Training samples: {len(train_files)}")
print(f"Validation samples: {len(val_files)}")

# Create TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_files, val_labels))

# Apply preprocessing
train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Add data augmentation for training
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.1),
    layers.RandomContrast(0.1)
])

def augment_image(image, label):
    return data_augmentation(image, training=True), label

train_ds = train_ds.map(augment_image)

# Batch and prefetch
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

## 5. Create and Train the Model

In [ ]:
# Create a MobileNetV2-based model for mobile deployment
def create_fire_detection_model():
    # Use MobileNetV2 as base model (pre-trained on ImageNet)
    base_model = keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model initially
    base_model.trainable = False
    
    # Add custom classification head
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    return model

# Create the model
model = create_fire_detection_model()

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Train the model
callbacks = [
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.2)
]

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks
)

## 6. Fine-tuning (Optional but Recommended)

In [ ]:
# Unfreeze the base model for fine-tuning
model.layers[0].trainable = True

# Use a lower learning rate for fine-tuning
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fine-tune for a few more epochs
fine_tune_epochs = 10
total_epochs = EPOCHS + fine_tune_epochs

history_fine = model.fit(
    train_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=val_ds,
    callbacks=callbacks
)

## 7. Evaluate Model Performance

In [ ]:
# Plot training history
def plot_training_history(history, history_fine=None):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Val Accuracy')
    
    if history_fine:
        ax1.plot(range(len(history.history['accuracy']), len(history.history['accuracy']) + len(history_fine.history['accuracy'])), 
                 history_fine.history['accuracy'], label='Fine-tune Train Accuracy')
        ax1.plot(range(len(history.history['val_accuracy']), len(history.history['val_accuracy']) + len(history_fine.history['val_accuracy'])), 
                 history_fine.history['val_accuracy'], label='Fine-tune Val Accuracy')
    
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Val Loss')
    
    if history_fine:
        ax2.plot(range(len(history.history['loss']), len(history.history['loss']) + len(history_fine.history['loss'])), 
                 history_fine.history['loss'], label='Fine-tune Train Loss')
        ax2.plot(range(len(history.history['val_loss']), len(history.history['val_loss']) + len(history_fine.history['val_loss'])), 
                 history_fine.history['val_loss'], label='Fine-tune Val Loss')
    
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

plot_training_history(history, history_fine)

In [ ]:
# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(val_ds, verbose=0)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

# Get predictions for confusion matrix
y_pred = model.predict(val_ds)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()
y_true = val_labels

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=['Non-Fire', 'Fire']))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Fire', 'Fire'], yticklabels=['Non-Fire', 'Fire'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## 8. Convert to TensorFlow Lite for Mobile

In [ ]:
# Convert model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: Apply optimizations for mobile deployment
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('fire_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

print(f"TensorFlow Lite model saved as 'fire_detection_model.tflite'")
print(f"Model size: {len(tflite_model) / 1024 / 1024:.2f} MB")

## 9. Test the TensorFlow Lite Model

In [ ]:
# Test the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:")
print(f"  Name: {input_details[0]['name']}")
print(f"  Shape: {input_details[0]['shape']}")
print(f"  Type: {input_details[0]['dtype']}")

print("\nOutput details:")
print(f"  Name: {output_details[0]['name']}")
print(f"  Shape: {output_details[0]['shape']}")
print(f"  Type: {output_details[0]['dtype']}")

# Test with a sample image
def test_tflite_model(image_path):
    # Load and preprocess image
    img = Image.open(image_path).convert('RGB')
    img = img.resize((IMG_SIZE, IMG_SIZE))
    img_array = np.array(img, dtype=np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], img_array)
    
    # Run inference
    interpreter.invoke()
    
    # Get output
    output = interpreter.get_tensor(output_details[0]['index'])
    confidence = output[0][0]
    
    return confidence, "Fire" if confidence > 0.5 else "No Fire"

# Test with sample images
sample_images = val_files[:5]
sample_labels = val_labels[:5]

for i, (img_path, true_label) in enumerate(zip(sample_images, sample_labels)):
    confidence, prediction = test_tflite_model(img_path)
    true_class = "Fire" if true_label == 1 else "No Fire"
    print(f"Image {i+1}: True={true_class}, Predicted={prediction}, Confidence={confidence:.4f}")

## 10. Save Model Files for Flutter Integration

In [ ]:
# Save the Keras model as well (for future reference)
model.save('fire_detection_model.h5')
print("Keras model saved as 'fire_detection_model.h5'")

# Create a model info file
model_info = {
    'model_name': 'Fire Detection Model',
    'input_shape': [1, IMG_SIZE, IMG_SIZE, 3],
    'output_shape': [1, 1],
    'input_type': 'float32',
    'output_type': 'float32',
    'classes': ['No Fire', 'Fire'],
    'threshold': 0.5,
    'preprocessing': 'Normalize to [0, 1] range',
    'validation_accuracy': float(val_accuracy),
    'model_size_mb': len(tflite_model) / 1024 / 1024
}

import json
with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("\nModel information saved to 'model_info.json'")
print(json.dumps(model_info, indent=2))

# List all files created
print("\nFiles created:")
!ls -la fire_detection_model.* model_info.json

## 11. Download Files

**Download these files to use in your Flutter app:**

1. **`fire_detection_model.tflite`** - The main model file for Flutter
2. **`model_info.json`** - Model configuration and metadata
3. **`fire_detection_model.h5`** - Full Keras model (for future training/improvements)

**For Flutter integration, you'll need:**
- Place `fire_detection_model.tflite` in `assets/models/` folder
- Add the tflite_flutter package to pubspec.yaml
- Use the model info for proper preprocessing in your app

In [ ]:
# Download files (uncomment the lines below in Google Colab)
# from google.colab import files
# files.download('fire_detection_model.tflite')
# files.download('model_info.json')
# files.download('fire_detection_model.h5')

print("Training complete! 🔥")
print(f"Final validation accuracy: {val_accuracy:.4f}")
print(f"Model size: {len(tflite_model) / 1024 / 1024:.2f} MB")
print("\nReady for Flutter integration!")